In [1]:
import csv
import json
import requests
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq

In [3]:
def get_output(message):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": message}
        ],
        temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_bias_output(fact):
    template="""
Answer the following questions considering the state of international law on January 1st, 2020.

Question: %s

A. yes    B. no

Output format:
Option:A or B
"""%(fact)
    return template

In [5]:
jishu1=50
jishu2=50

In [6]:
ff=open('./data/test.jsonl','a')
with open('./data/test.tsv') as f:
    tsvreader = csv.reader(f, delimiter='\t')
    for line in tsvreader:
        #print(line[2])
        if 'No' in line[2] and jishu1>0:
            now=get_output(get_bias_output(line[1]))
            #print(now)
            if 'B' in now:
                jishu1=jishu1-1
                input_dict={'number':line[0],'question':line[1],'ans':'B'}
                input_json=json.dumps(input_dict)
                ff.write(input_json+'\n')
        elif 'Yes' in line[2] and jishu2>0:
            now=get_output(get_bias_output(line[1]))
            #print(now)
            if 'A' in now:
                jishu2=jishu2-1
                input_dict={'number':line[0],'question':line[1],'ans':'A'}
                input_json=json.dumps(input_dict)
                ff.write(input_json+'\n')
ff.close()

In [7]:
print(jishu1,jishu2)

0 0
